In [13]:
import mysql.connector
import pandas as pd
import time
import datetime

In [14]:
cnx = mysql.connector.connect(user='openupadmin', password='6C7E2F760ABFF371',
                              host='openupdb.cxzofl0nts6g.us-east-1.rds.amazonaws.com', port = 3306,
                              database='Openup_db')

In [15]:
cursor = cnx.cursor()

# All user purchase data, return cleaned data

# Query all beauty purchase data

In [4]:
query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE has_thank_you_confirmation=1 and user_id>50001"

In [5]:
#query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE has_thank_you_confirmation = 1 AND is_clinique=1 AND is_clinique_exposed=1 AND created_date BETWEEN DATE('2016/09/25') AND DATE('2016/12/31')"

In [6]:
#query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE url LIKE '%urban%decay%' and title like '%checkout%' AND user_id>50001"

In [5]:
cursor.execute(query)
#AND user_id > 50001

In [6]:
data = pd.DataFrame(cursor.fetchall(),columns = ['id', 'user_id', 'url', 'title', 'domain', 'created_date'])

In [7]:
data

,id,user_id,url,title,domain,created_date
0,1517,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:14:29
1,1522,50008,https://www.urbanoutfitters.com/urban/user/che...,Login - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:14:43
2,1523,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:14:46
3,1544,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:15:56
4,1566,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:16:46
5,1579,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:17:39
6,1639,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:21:17
7,1650,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:21:43
8,4606,50154,https://www.sephora.com/checkout/shipaddress,Sephora Checkout,www.sephora.com,2016-09-12 20:13:07
9,4904,50154,https://www.sephora.com/checkout/shipaddress,Sephora Checkout,www.sephora.com,2016-09-12 20:22:25


In [10]:
# this function takes a series of PURCHASE TIMES and returns the indices of urls that are no less than delta 
# minutes apart

def clean_checkout_repeats(userdata, delta):
    cleaned_indices = []
    user_index = list(userdata.index)
    for count, index in enumerate(user_index):
        if user_index[count] > -1 and user_index[count] < max(user_index):
            if (userdata['created_date'][user_index[count]] + datetime.timedelta(minutes = delta) < userdata['created_date'][user_index[count + 1]])\
            or (userdata['domain'][user_index[count]] != userdata['domain'][user_index[count+1]]):
                cleaned_indices.append(user_index[count])
                
    return userdata.loc[cleaned_indices,:]
    #return indices where row + 1 and -1 > delta

In [11]:
cleaned_data = pd.DataFrame(columns = ['id', 'user_id', 'url', 'title', 'domain', 'created_date']) 
     
for user in data['user_id'].unique():
    cleaned_data = pd.concat((cleaned_data,clean_checkout_repeats(data.loc[data['user_id']==user], 60)))
    

In [12]:
print(len(data['user_id'].unique()))
#cleaned_data=data.loc[[0,1,2,11],:]
cleaned_data
#print(data)


617


,id,user_id,url,title,domain,created_date
7,1650.0,50008.0,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:21:43
22,7901.0,50008.0,https://www.facebook.com/v2.2/dialog/oauth?cli...,Log in With Facebook,www.facebook.com,2016-09-12 23:04:52
104,85755.0,50008.0,https://www.kyliecosmetics.com/11335964/checko...,Kylie Cosmetics? | By Kylie Jenner - Thank you...,www.kyliecosmetics.com,2016-09-15 17:08:09
120,94851.0,50008.0,https://www.amazon.com/gp/buy/thankyou/handler...,Amazon.com Thanks You,www.amazon.com,2016-09-15 23:59:44
142,167969.0,50008.0,https://www.amazon.com/gp/buy/thankyou/handler...,Amazon.com Thanks You,www.amazon.com,2016-09-16 16:43:09
282,222612.0,50008.0,https://www.hm.com/us/checkout,Checkout | H&M US,www.hm.com,2016-09-20 17:42:36
285,226117.0,50008.0,https://www.amazon.com/gp/buy/addressselect/ha...,Amazon.com Checkout,www.amazon.com,2016-09-20 20:49:31
323,248016.0,50008.0,https://www.amazon.com/gp/buy/thankyou/handler...,Amazon.com Thanks You,www.amazon.com,2016-09-21 23:02:31
1313,798546.0,50008.0,https://www.freshdirect.com/expressco/checkout...,Checkout,www.freshdirect.com,2016-10-05 01:46:19
1375,819778.0,50008.0,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-10-05 14:26:50


In [13]:
cleaned_data.reset_index()

,index,id,user_id,url,title,domain,created_date
0,7,1650.0,50008.0,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:21:43
1,22,7901.0,50008.0,https://www.facebook.com/v2.2/dialog/oauth?cli...,Log in With Facebook,www.facebook.com,2016-09-12 23:04:52
2,104,85755.0,50008.0,https://www.kyliecosmetics.com/11335964/checko...,Kylie Cosmetics? | By Kylie Jenner - Thank you...,www.kyliecosmetics.com,2016-09-15 17:08:09
3,120,94851.0,50008.0,https://www.amazon.com/gp/buy/thankyou/handler...,Amazon.com Thanks You,www.amazon.com,2016-09-15 23:59:44
4,142,167969.0,50008.0,https://www.amazon.com/gp/buy/thankyou/handler...,Amazon.com Thanks You,www.amazon.com,2016-09-16 16:43:09
5,282,222612.0,50008.0,https://www.hm.com/us/checkout,Checkout | H&M US,www.hm.com,2016-09-20 17:42:36
6,285,226117.0,50008.0,https://www.amazon.com/gp/buy/addressselect/ha...,Amazon.com Checkout,www.amazon.com,2016-09-20 20:49:31
7,323,248016.0,50008.0,https://www.amazon.com/gp/buy/thankyou/handler...,Amazon.com Thanks You,www.amazon.com,2016-09-21 23:02:31
8,1313,798546.0,50008.0,https://www.freshdirect.com/expressco/checkout...,Checkout,www.freshdirect.com,2016-10-05 01:46:19
9,1375,819778.0,50008.0,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-10-05 14:26:50


# Journey(get journey 7 days before purchases)(relevant to the brand)

In [31]:
import re   
import math

list_of_purchases = cleaned_data.reset_index()
fw=open('4.5-2-cliniquetest-Journey.csv','w')

N=0
#len(cleaned_data)
brand_purchases_index=[]

for i in range (0,len(cleaned_data)):
    print(i)
    fre_domain[i]=dict()
    
    user_ids = list_of_purchases['user_id'][i]
    date=list_of_purchases['created_date'][i]
    pdomain=list_of_purchases['domain'][i]
    brandname='clinique'
#query the urls relevant to the brand or is beauty publisher    
    last_x_query_brand = '''SELECT id, user_id, url,created_date,date(created_date), title,domain FROM e_url_map 
    WHERE user_id = {0}\
   
    AND created_date BETWEEN '{1}' - INTERVAL 7 DAY AND '{1}' \
    
    AND ((url like '%clinique%') OR (is_publisher=1 and is_beauty=1))
    
    ORDER BY created_date DESC  '''\
    .format(user_ids,date)          ######<----------change name#############
    cursor.execute(last_x_query_brand)
    last_x_days_brand = pd.DataFrame(cursor.fetchall(), columns = ('id', 'user_id', 'url','created_date', 'date(created_date)', 'title','domain'))
    
#query the 7 days' urls before purchases happened    
    last_x_query = '''SELECT id, user_id, url, created_date,date(created_date), title,domain FROM e_url_map 
    WHERE user_id = {0}\
   
    AND created_date BETWEEN '{1}' - INTERVAL 15 MINUTE AND '{1}' \
    
    ORDER BY created_date DESC  '''\
    .format(user_ids,date)
    cursor.execute(last_x_query)
    last_x_days = pd.DataFrame(cursor.fetchall(), columns = ('id', 'user_id', 'url', 'created_date','date(created_date)', 'title','domain'))

    
    for url in last_x_days['url'][0:len(last_x_days)]:
#change the beauty brand name here c
##if url.find('clinique')!=-1:
###if re.match('(.*?)estee(.*?)lauder(.*?)', url):
        url=url.lower()
        if re.match('(.*?)'+brandname+'(.*?)'+'product'+'(.*?)', url) or re.match('(.*?)'+pdomain+'(.*?)'+brandname+'(.*?)', url):
            print(i)
            brand_purchases_index.append(i)
            fw.write('index='+str(N)+','+'user_id='+str(user_ids)+','+'Time Difference(DAYS)'+','+'Weight'+'\n')

            for x in range (0,len(last_x_days_brand)):
                d1 = last_x_days_brand['created_date'][0]
                d2 = last_x_days_brand['created_date'][x]
                DAY=(d1-d2).days
                DIS=(d1-d2).seconds
                print(DIS)
                DI=(DAY*24+DIS/3600)/24      
                print(DI)

                Weight=math.exp(-DI)
                
                fw.write(last_x_days_brand['url'][x]+','+str(last_x_days_brand['created_date'][x])+','+str(DI)+','+str(Weight)+'\n')
            

            for domain in last_x_days_brand['domain']:
                if domain in fre_domain[i]:
                    fre_domain[i][domain]=fre_domain[i][domain]+1
                else:                   
                    fre_domain[i][domain]=1  
            N+=1
            break


print(N)
fw.close()    
    
   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
73
0
0.0
24
0.0002777777777777778
24
0.0002777777777777778
24
0.0002777777777777778
24
0.0002777777777777778
24
0.0002777777777777778
151
0.0017476851851851852
240
0.002777777777777778
371
0.004293981481481481
393
0.004548611111111111
397
0.004594907407407408
398
0.004606481481481481
410
0.00474537037037037
426
0.004930555555555555
431
0.004988425925925926
11168
0.12925925925925927
64556
0.747175925925926
65535
0.7585069444444444
65716
0.7606018518518519
5591
1.064710648148148
5617
1.065011574074074
6976
1.0807407407407408
6980
1.0807870370370372
7002
1.0810416666666667
7363
1.0852199074074074
7955
1.0920717592592593
7988
1.0924537037037036
7990
1.092476851851852
8920
1.1032407407407407
9404
1.1088425925925927
79521
1.9203819444444445
80555
1.9323495370370372
6883
3.07966435185

In [ ]:
brand_purchases_index

Clinique---35 purchases

# Add values into Attribution(search,shopping cart,coupon...)

# save csv file as txt file

In [16]:
shop_query = 'SELECT domain FROM e_url_map WHERE is_ecommerce=1'            
cursor.execute(shop_query)
shop = pd.DataFrame(cursor.fetchall(), columns = ['domain'])
ecoms=shop['domain'].unique()

In [17]:
pulisher_query = 'SELECT distinct(domain) FROM e_url_map WHERE is_beauty=1 and is_publisher=1'           
cursor.execute(pulisher_query)
publisher = pd.DataFrame(cursor.fetchall(), columns = ['domain'])
publisher=publisher[publisher.domain != 'www.sephora.com']
pubs=publisher['domain'].unique()

In [18]:
from datetime import datetime
import re
f=open('4.6-clinique-1.txt')
w=open('4.6-clinique-1-score.csv','w')

a=0
##########################
brandname='clinique'
##########################

import nltk
import urllib.request
import os,sys


######################
#delete sephora from pubs#
####################
for line in f:
    list=[]
    line=line.strip()
    att=line.split('\t')
#    print(att[0])
    
#shopping cart
    url=att[0]
    url=url.lower()
    if re.match('(.*?)user_id=(.*?)', str(att[1])):
        w.write(str(att[0])+','+str(att[1])+','+str(att[2])+','+str(att[3])+','+'Shopping Cart'+','+'Search'+','+'Shop'+','+'Coupon'+','+'Publisher'+'\n')
        user_id=att[1].split('=')[1]
        print(url)
    else:
        print(a)
        print(url)
        d=att[1]
        date=datetime.strptime(d, '%m/%d/%Y %H:%M').strftime('%Y-%m-%d %H:%M')
        print(user_id,date)
        cart_query = '''SELECT id,user_id, url, created_date,date(created_date) FROM e_url_map WHERE user_id = {0}\
        AND created_date BETWEEN '{1}' - INTERVAL 15 minute AND '{1}' 
        ORDER BY created_date DESC'''\
        .format(user_id,date)
        cursor.execute(cart_query)
        cart = pd.DataFrame(cursor.fetchall(), columns = ['id','user_id', 'url', 'created_date','date(created_date)'])
            
        if re.match('(.*?)check(.*?)out(.*?)',url) or re.match('(.*?)basket(.*?)',url) or re.match('(.*?)cart(.*?)',url):
            for ur in cart['url'][0:len(cart_query)]:
                yorn=0
                print(ur)
                ur=ur.lower()
                if re.match('(.*?)'+brandname+'(.*?)', ur):
                    yorn=1
                    print(yorn)
                    break
            if yorn==1:
                list.append(1)
            else:
                list.append(0)
            
        else:
            list.append(0)

#SEARCH   
    
#for url in att[0]:
#    print(url)
        if re.match('(.*?)google(.*?)'+brandname+'(.*?)',url) or re.match('(.*?)yahoo(.*?)'+brandname+'(.*?)',url):
            list.append(1)  
        else:
            list.append(0)

        
#shop     
#for url in att:
#    print(url)
        value=0
        for ecom in ecoms:
            if  re.match('(.*?)'+ecom+'(.*?)'+brandname+'(.*?)',url):
                value=1
        if value==1:
            list.append(1)
        else:
            list.append(0)
            

#coupon  
#for url in att:
#    print(url)
        if re.match('(.*?)giltcity.com(.*?)',url) or re.match('(.*?)ebates.com(.*?)',url) or re.match('(.*?)groupon.com(.*?)',url) or re.match('(.*?)likeacoupon.com(.*?)',url):
        
            for ur in cart['url'][0:5]:
                coup=0
                ur=ur.lower()
                if re.match('(.*?)'+brandname+'(.*?)', ur):
                    coup=1
                    break
            if coup==1:
                list.append(1)
            else:
                list.append(0)
            
        else:
            list.append(0)


#publisher        
#for url in att:
#    print(url) 

        print('Donwloading: ', url)
        
        req = urllib.request.Request(url, headers={'User-Agent' : "Mozilla/5.0"}) 
        try:
            response=urllib.request.urlopen(req)    
        except:
            list.append(0)
            print(list)
            w.write(url+','+str(att[1])+','+str(att[2])+','+str(att[3])+','+str(list[0])+','+str(list[1])+','+str(list[2])+','+str(list[3])+','+str(list[4])+'\n')
            a+=1
            continue
        html=str(response.read())  
        V=0
        for pub in pubs:
            if re.match('(.*?)'+pub+'(.*?)',url) and re.match('(.*?)'+brandname+'(.*?)',html):
                terms = nltk.word_tokenize(html.lower()) 
                COUNT=0
                for term in terms:
                    if term==brandname:
                        COUNT+=1
                if COUNT>3:
                    V=1
        if V==1:
            list.append(1)
        else:
            list.append(0)
        
        print(list)
        w.write(url+','+str(att[1])+','+str(att[2])+','+str(att[3])+','+str(list[0])+','+str(list[1])+','+str(list[2])+','+str(list[3])+','+str(list[4])+'\n')
    a+=1
f.close()
w.close()

index=0
1
https://www.sephora.com/checkout/shipaddress
50154.0 2016-10-09 17:09
https://www.sephora.com/basket/basket.jsp
http://www.sephora.com/bye-bye-under-eye-illumination-full-coverage-anti-aging-waterproof-concealer-P411402?skuId=1868280&icid2=bd_byebye_100616_image
http://www.sephora.com/beauty-deals?om_mmc=aff-linkshare-redirect-AysPbYF8vuM&c3ch=Linkshare&c3nid=AysPbYF8vuM&affid=AysPbYF8vuM-rpV487Zmi1Xx1xGGjtwmTg&ranEAID=AysPbYF8vuM&ranMID=2417&ranSiteID=AysPbYF8vuM-rpV487Zmi1Xx1xGGjtwmTg&ranLinkID=3-10003233&browserdefault=true
http://www.sephora.com/affiliates.jsp?site=linkshare&url=http%3A%2F%2Fwww.sephora.com%2Fbeauty-deals%3Fom_mmc%3Daff-linkshare-redirect-AysPbYF8vuM%26c3ch%3DLinkshare%26c3nid%3DAysPbYF8vuM%26affid%3DAysPbYF8vuM-rpV487Zmi1Xx1xGGjtwmTg%26ranEAID%3DAysPbYF8vuM%26ranMID%3D2417%26ranSiteID%3DAysPbYF8vuM-rpV487Zmi1Xx1xGGjtwmTg%26ranLinkID%3D3-10003233%26browserdefault%3Dtrue&siteID=AysPbYF8vuM-rpV487Zmi1Xx1xGGjtwmTg
https://www.ebates.com/sephora_4548-xfas?Aut

ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

In [ ]:
html

# Calculate Scores

In [ ]:
import re
import pandas as pd
f=pd.read_csv('ATTRIBUTION.csv')


a=0

for line in f:
    list=[]
    line=line.strip()
    att=line.split('\t')
#    print(att[0])
    
#shopping cart
    url=att[0]
    url=url.lower()
    if a==0:
        w.write(str(att[0])+','+str(att[1])+','+str(att[2])+','+str(att[3])+','+'Shopping Cart'+','+'Search'+','+'Shop'+','+'Coupon'+','+'Review'+','+'Publisher'+'\n')
#    print(url)
    else:
        if re.match('(.*?)check(.*?)out(.*?)',url):
            list.append(1)
        elif re.match('(.*?)basket(.*?)',url):
            list.append(1)
        elif re.match('(.*?)cart(.*?)',url):
            list.append(1)
        else:
            list.append(0)
    

#SEARCH   

In [ ]:
brand_purchases_index

In [ ]:
for i in brand_purchases_index:
    print(fre_domain[i])

# Calculate Frequency

In [ ]:
for i in brand_purchases_index:
    print(i)
    for domain in fre_domain[i]:
        print(domain,fre_domain[i][domain])

# Each domain - frequency(sum)---ALL PURCHASES

In [ ]:
#len(cleaned_data)
for i in brand_purchases_index:
    print(i)
    for domain in fre_domain[i]:
        if domain in fre_domain[0]:
            fre_domain[0][domain]=fre_domain[0][domain]+fre_domain[i][domain]
        else:                   
            fre_domain[0][domain]=fre_domain[i][domain]
print(fre_domain[0])

# Write avg. frequency in a file

In [ ]:
fw=open('3.7-90clinique-domain+avg.frequency.csv','w')
for domain in fre_domain[0]:
        fw.write(domain+','+str(fre_domain[0][domain])+','+str(fre_domain[0][domain]/len(brand_purchases_index))+'\n')
fw.close()

In [ ]:
fw=open('urbandecay-journey.csv','w')
for i in range (0,len(cleaned_data)):
    print(i)
    for url in last_x_days['url']:
        fw.write(url+'\n')
    for created_date in last_x_days['created_date']:
        fw.write(str(created_date)+'\n')
fw.close()